In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.svm import SVC,LinearSVC
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import warnings
warnings.filterwarnings('ignore')


In [2]:
# Load the csv file into a pandas DataFrame
df = pd.read_csv('tfidf_data.csv')

In [3]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,type
0,3,2,3,5,3,2,2,2,2,1,...,0.042567,0.263627,0.222827,0.340988,0.084534,0.117008,0.040395,0.129523,0.163540,ENFP
1,5,4,5,4,5,4,4,4,1,3,...,0.148636,0.043835,0.216131,0.264593,0.049196,0.090793,0.141052,0.000000,0.095175,ENFP
2,2,3,4,3,3,5,2,5,4,2,...,0.052480,0.139294,0.320505,0.140132,0.052110,0.192342,0.000000,0.000000,0.201625,ENFP
3,3,2,5,5,5,1,5,4,5,5,...,0.246176,0.043561,0.343646,0.043823,0.048888,0.225563,0.046723,0.000000,0.283739,ENFP
4,2,2,3,3,1,2,1,4,3,1,...,0.190135,0.168222,0.221180,0.000000,0.062932,0.058071,0.000000,0.257131,0.182623,ENFP
5,3,1,5,3,3,4,4,2,2,4,...,0.198344,0.263227,0.129785,0.176540,0.000000,0.227170,0.047056,0.050293,0.047627,ENFP
6,5,1,3,5,2,1,4,4,1,4,...,0.107631,0.142839,0.093903,0.095799,0.053436,0.049309,0.051070,0.218333,0.103378,ENFP
7,3,5,2,4,2,3,1,3,1,3,...,0.000000,0.216813,0.171041,0.174493,0.000000,0.179629,0.046511,0.099421,0.094149,ENFP
8,4,3,4,2,1,3,1,1,5,1,...,0.106263,0.094016,0.185419,0.141872,0.105514,0.340777,0.201682,0.269447,0.051032,ENFP
9,2,4,1,4,2,4,1,3,2,4,...,0.090301,0.079894,0.118176,0.241123,0.089664,0.206849,0.085694,0.228973,0.086733,ENFP


In [4]:
df.describe(include='O')

,type
count,52002
unique,16
top,ENTP
freq,9504


In [5]:
df['type'].value_counts()

ENTP    9504
INFP    9470
ENFP    6167
INTP    5338
INFJ    5242
INTJ    3052
ISTP    2963
ENTJ    2955
ESTP    1986
ENFJ    1534
ISTJ    1243
ISFP     875
ISFJ     650
ESTJ     482
ESFP     360
ESFJ     181
Name: type, dtype: int64

In [6]:
# Split the data into training and testing sets
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42, stratify=df.type)

# Extract the feature columns from the training data and store it in a numpy array
train_post = train_data.iloc[:, :-1].values
test_post = test_data.iloc[:, :-1].values

# Extract the target column from the training data and store it in a numpy array
train_target = train_data.iloc[:, -1].values

# Transform the target values from categorical to numerical values
target_encoder = LabelEncoder()
train_target = target_encoder.fit_transform(train_target)

# Extract the target column from the test data and store it in a numpy array
test_target = test_data.iloc[:, -1].values

# Transform the test target values from categorical to numerical values
test_target = target_encoder.transform(test_target)

## Model Selection ##

In [7]:
models_accuracy={}

### Logistic Regression ###

In [8]:
model_log=LogisticRegression(max_iter=3000,C=0.5,n_jobs=-1)
model_log.fit(train_post,train_target)

LogisticRegression(C=0.5, max_iter=3000, n_jobs=-1)

In [9]:
print('Train classification report \n ',classification_report(train_target,model_log.predict(train_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))
print('Test classification report \n',classification_report(test_target,model_log.predict(test_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))

Train classification report 
                precision    recall  f1-score   support

        ENFJ       0.75      0.69      0.72      1227
        ENFP       0.82      0.82      0.82      4933
        ENTJ       0.68      0.62      0.65      2364
        ENTP       0.76      0.83      0.79      7603
        ESFJ       0.78      0.46      0.58       145
        ESFP       0.72      0.47      0.57       288
        ESTJ       0.80      0.69      0.75       386
        ESTP       0.69      0.63      0.66      1589
        INFJ       0.76      0.78      0.77      4194
        INFP       0.83      0.86      0.85      7576
        INTJ       0.64      0.62      0.63      2442
        INTP       0.67      0.66      0.67      4270
        ISFJ       0.69      0.55      0.62       520
        ISFP       0.73      0.61      0.66       700
        ISTJ       0.68      0.64      0.66       994
        ISTP       0.67      0.66      0.66      2370

    accuracy                           0.75     4

In [10]:
models_accuracy['logistic regression']=accuracy_score(test_target,model_log.predict(test_post))

### Linear Support Vector Classifier ###

In [11]:
model_linear_svc=LinearSVC(C=0.1)
model_linear_svc.fit(train_post,train_target)

LinearSVC(C=0.1)

In [12]:
print('Train classification report \n ',classification_report(train_target,model_linear_svc.predict(train_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))
print('Test classification report \n',classification_report(test_target,model_linear_svc.predict(test_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))

Train classification report 
                precision    recall  f1-score   support

        ENFJ       0.74      0.62      0.67      1227
        ENFP       0.79      0.80      0.79      4933
        ENTJ       0.67      0.47      0.55      2364
        ENTP       0.69      0.87      0.77      7603
        ESFJ       0.94      0.31      0.47       145
        ESFP       0.75      0.37      0.50       288
        ESTJ       0.84      0.69      0.75       386
        ESTP       0.71      0.51      0.60      1589
        INFJ       0.71      0.77      0.74      4194
        INFP       0.78      0.87      0.82      7576
        INTJ       0.64      0.53      0.58      2442
        INTP       0.65      0.60      0.62      4270
        ISFJ       0.69      0.39      0.50       520
        ISFP       0.75      0.48      0.59       700
        ISTJ       0.69      0.55      0.61       994
        ISTP       0.68      0.60      0.64      2370

    accuracy                           0.72     4

In [13]:
models_accuracy['Linear Support Vector classifier']=accuracy_score(test_target,model_linear_svc.predict(test_post))

### Support Vector classifier

In [14]:
model_svc=SVC()
model_svc.fit(train_post,train_target)

SVC()

In [15]:

print('Train classification report \n ',classification_report(train_target,model_svc.predict(train_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))
print('Test classification report \n ',classification_report(test_target,model_svc.predict(test_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))

Train classification report 
                precision    recall  f1-score   support

        ENFJ       0.73      0.44      0.55      1227
        ENFP       0.78      0.76      0.77      4933
        ENTJ       0.65      0.59      0.62      2364
        ENTP       0.71      0.85      0.77      7603
        ESFJ       0.80      0.06      0.10       145
        ESFP       0.75      0.04      0.08       288
        ESTJ       0.89      0.28      0.43       386
        ESTP       0.68      0.52      0.59      1589
        INFJ       0.70      0.78      0.74      4194
        INFP       0.77      0.86      0.81      7576
        INTJ       0.63      0.60      0.61      2442
        INTP       0.68      0.65      0.66      4270
        ISFJ       0.74      0.27      0.40       520
        ISFP       0.76      0.28      0.40       700
        ISTJ       0.64      0.53      0.58       994
        ISTP       0.63      0.67      0.65      2370

    accuracy                           0.71     4

In [16]:
models_accuracy['Support Vector classifier']=accuracy_score(test_target,model_svc.predict(test_post))

### Multinomial Naive Bayes

In [17]:
model_multinomial_nb=MultinomialNB()
model_multinomial_nb.fit(train_post,train_target)

MultinomialNB()

In [18]:

print('Train classification report \n ',classification_report(train_target,model_multinomial_nb.predict(train_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))
print('Test classification report \n ',classification_report(test_target,model_multinomial_nb.predict(test_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))

Train classification report 
                precision    recall  f1-score   support

        ENFJ       0.89      0.05      0.10      1227
        ENFP       0.78      0.52      0.63      4933
        ENTJ       0.71      0.20      0.31      2364
        ENTP       0.49      0.95      0.64      7603
        ESFJ       0.00      0.00      0.00       145
        ESFP       0.00      0.00      0.00       288
        ESTJ       1.00      0.09      0.17       386
        ESTP       0.71      0.07      0.13      1589
        INFJ       0.63      0.61      0.62      4194
        INFP       0.56      0.94      0.70      7576
        INTJ       0.67      0.33      0.45      2442
        INTP       0.62      0.42      0.50      4270
        ISFJ       0.93      0.03      0.05       520
        ISFP       0.70      0.01      0.02       700
        ISTJ       0.83      0.18      0.29       994
        ISTP       0.63      0.42      0.50      2370

    accuracy                           0.57     4

In [19]:
models_accuracy['Multinomial Naive Bayes']=accuracy_score(test_target,model_multinomial_nb.predict(test_post))

### Decision Tree Classifier

In [20]:
model_tree=DecisionTreeClassifier(max_depth=14)
model_tree.fit(train_post,train_target)

DecisionTreeClassifier(max_depth=14)

In [21]:

print('Train classification report \n ',classification_report(train_target,model_tree.predict(train_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))
print('Test classification report \n ',classification_report(test_target,model_tree.predict(test_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))

Train classification report 
                precision    recall  f1-score   support

        ENFJ       0.63      0.70      0.66      1227
        ENFP       0.84      0.86      0.85      4933
        ENTJ       0.68      0.68      0.68      2364
        ENTP       0.79      0.91      0.84      7603
        ESFJ       0.84      0.57      0.68       145
        ESFP       0.69      0.42      0.52       288
        ESTJ       0.81      0.67      0.74       386
        ESTP       0.71      0.64      0.68      1589
        INFJ       0.83      0.82      0.82      4194
        INFP       0.86      0.89      0.88      7576
        INTJ       0.76      0.67      0.71      2442
        INTP       0.78      0.79      0.79      4270
        ISFJ       0.82      0.57      0.67       520
        ISFP       0.82      0.51      0.63       700
        ISTJ       0.76      0.57      0.65       994
        ISTP       0.81      0.68      0.74      2370

    accuracy                           0.80     4

In [22]:
models_accuracy['Decision Tree classifier']=accuracy_score(test_target,model_tree.predict(test_post))

### Random Forest Classifier

In [23]:
model_forest=RandomForestClassifier(max_depth=10)
model_forest.fit(train_post,train_target)

RandomForestClassifier(max_depth=10)

In [24]:

print('Train classification report \n ',classification_report(train_target,model_forest.predict(train_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))
print('Test classification report \n ',classification_report(test_target,model_forest.predict(test_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))

Train classification report 
                precision    recall  f1-score   support

        ENFJ       0.97      0.28      0.44      1227
        ENFP       0.85      0.84      0.85      4933
        ENTJ       0.99      0.06      0.11      2364
        ENTP       0.51      0.98      0.67      7603
        ESFJ       1.00      0.01      0.01       145
        ESFP       1.00      0.02      0.03       288
        ESTJ       1.00      0.28      0.44       386
        ESTP       1.00      0.15      0.27      1589
        INFJ       0.79      0.81      0.80      4194
        INFP       0.69      0.94      0.79      7576
        INTJ       0.93      0.27      0.41      2442
        INTP       0.63      0.73      0.68      4270
        ISFJ       1.00      0.09      0.16       520
        ISFP       1.00      0.02      0.04       700
        ISTJ       1.00      0.07      0.13       994
        ISTP       0.97      0.34      0.51      2370

    accuracy                           0.66     4

In [25]:
models_accuracy['Random Forest Classifier']=accuracy_score(test_target,model_forest.predict(test_post))

### XGBoost Classifier

In [26]:
model_xgb = xgb.XGBClassifier(tree_method='gpu_hist', max_depth=5, n_estimators=50, learning_rate=0.1)
model_xgb.fit(train_post, train_target)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=50, n_jobs=None, num_parallel_tree=None,
              objective='multi:softprob', predictor=None, ...)

In [27]:

print('Train classification report \n ',classification_report(train_target,model_xgb.predict(train_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))
print('Test classification report \n ',classification_report(test_target,model_xgb.predict(test_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))

Train classification report 
                precision    recall  f1-score   support

        ENFJ       0.93      0.55      0.69      1227
        ENFP       0.90      0.87      0.89      4933
        ENTJ       0.85      0.56      0.68      2364
        ENTP       0.76      0.97      0.85      7603
        ESFJ       0.99      0.50      0.67       145
        ESFP       0.95      0.56      0.71       288
        ESTJ       0.98      0.70      0.81       386
        ESTP       0.91      0.61      0.73      1589
        INFJ       0.88      0.90      0.89      4194
        INFP       0.86      0.96      0.90      7576
        INTJ       0.88      0.69      0.77      2442
        INTP       0.78      0.87      0.83      4270
        ISFJ       0.88      0.76      0.81       520
        ISFP       0.96      0.53      0.69       700
        ISTJ       0.88      0.65      0.75       994
        ISTP       0.80      0.77      0.79      2370

    accuracy                           0.84     4

In [28]:
models_accuracy['XGBoost Classifier']=accuracy_score(test_target,model_xgb.predict(test_post))

### CatBoost Classifier

In [29]:
model_cat=CatBoostClassifier(loss_function='MultiClass',eval_metric='MultiClass',task_type='GPU',verbose=False)
model_cat.fit(train_post,train_target)

In [30]:

print('train classification report \n ',classification_report(train_target,model_cat.predict(train_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))
print('test classification report \n ',classification_report(test_target,model_cat.predict(test_post),target_names=target_encoder.inverse_transform([i for i in range(16)])))

train classification report 
                precision    recall  f1-score   support

        ENFJ       1.00      1.00      1.00      1227
        ENFP       1.00      1.00      1.00      4933
        ENTJ       1.00      1.00      1.00      2364
        ENTP       1.00      1.00      1.00      7603
        ESFJ       1.00      1.00      1.00       145
        ESFP       1.00      1.00      1.00       288
        ESTJ       1.00      1.00      1.00       386
        ESTP       1.00      1.00      1.00      1589
        INFJ       1.00      1.00      1.00      4194
        INFP       1.00      1.00      1.00      7576
        INTJ       1.00      1.00      1.00      2442
        INTP       1.00      1.00      1.00      4270
        ISFJ       1.00      1.00      1.00       520
        ISFP       1.00      1.00      1.00       700
        ISTJ       1.00      1.00      1.00       994
        ISTP       1.00      1.00      1.00      2370

    accuracy                           1.00     4

In [31]:
models_accuracy['CatBoost Classifier']=accuracy_score(test_target,model_cat.predict(test_post))

### Model Accuracy Comparison 

In [32]:
models_accuracy

{'logistic regression': 0.7443515046630131,
 'Linear Support Vector classifier': 0.7176233054513989,
 'Support Vector classifier': 0.665032208441496,
 'Multinomial Naive Bayes': 0.5682145947505047,
 'Decision Tree classifier': 0.4111143159311605,
 'Random Forest Classifier': 0.5564849533698683,
 'XGBoost Classifier': 0.7606960869147198,
 'CatBoost Classifier': 0.9822132487260841}

In [33]:
models_accuracy.keys()

dict_keys(['logistic regression', 'Linear Support Vector classifier', 'Support Vector classifier', 'Multinomial Naive Bayes', 'Decision Tree classifier', 'Random Forest Classifier', 'XGBoost Classifier', 'CatBoost Classifier'])

In [34]:
accuarcy=pd.DataFrame(models_accuracy.items(),columns=['Models','Test accuracy'])
accuarcy['Test accuracy'] = accuarcy['Test accuracy'] * 100
accuarcy.sort_values(by='Test accuracy', ascending=False, ignore_index=True).style.background_gradient(cmap='Greens')

,Models,Test accuracy
0,CatBoost Classifier,98.221325
1,XGBoost Classifier,76.069609
2,logistic regression,74.435150
3,Linear Support Vector classifier,71.762331
4,Support Vector classifier,66.503221
5,Multinomial Naive Bayes,56.821459
6,Random Forest Classifier,55.648495
7,Decision Tree classifier,41.111432


### Model Saving and Loading

In [3]:
import pickle

In [36]:
# Save the logistic model to a file
with open('Models/model_log.pkl', 'wb') as file:
    pickle.dump(model_log, file)

# Save the linear support vector classifier model to a file
with open('Models/model_linear_svc.pkl', 'wb') as file:
    pickle.dump(model_linear_svc, file)

# Save the support vector model to a file
with open('Models/model_svc.pkl', 'wb') as file:
    pickle.dump(model_svc, file)

# Save the Multinomial Naive Bayes classifier model to a file
with open('Models/model_multinomial_nb.pkl', 'wb') as file:
    pickle.dump(model_multinomial_nb, file)

In [37]:
# Save the Decision Tree classifier model to a file
with open('Models/model_tree.pkl', 'wb') as file:
    pickle.dump(model_tree, file)

# Save the Random Forest Classifier classifier model to a file
with open('Models/model_forest.pkl', 'wb') as file:
    pickle.dump(model_forest, file)

# Save the catboost model to a file
with open('Models/model_cat.pkl', 'wb') as file:
    pickle.dump(model_cat, file)

# Save the xgboost classifier model to a file
with open('Models/model_xgb.pkl', 'wb') as file:
    pickle.dump(model_xgb, file)

In [4]:
# Load the model from the file
with open('Models/model_cat.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

TEST CODE BELOW NOT FOR DEPLOYMENT


### MBTI personality type Prediction

In [5]:
# Load the trained model
model = loaded_model

In [6]:

print("Welcome to the personality quiz!\n")
# Define the list of questions
questions = [
    "Question 1: I enjoy being in large groups of people.",
    "Question 2: I feel energized after interacting with many people.",
    "Question 3: I find it easy to approach and talk to strangers.",
    "Question 4: I like being the center of attention in social situations.",
    "Question 5: I prefer spending time alone over being with others.",
    "Question 6: I feel drained after spending a lot of time around people and need alone time to recharge.",
    "Question 7: I prefer to work independently and avoid group projects if possible.	",
    "Question 8: I prefer to listen and observe in social situations, rather than being the one to initiate conversation.",
    "Question 9: I prefer to focus on the facts and concrete information when making decisions.",
    "Question 10: I am more concerned with the present moment and practical realities than future possibilities.",
    "Question 11: I rely on past experiences to guide my decisions and actions.",
    "Question 12: I often see patterns and connections in things that others don't.",
    "Question 13: I trust my instincts and rely on my gut feeling when making decisions.",
    "Question 14: I am highly imaginative and enjoy exploring new ideas and possibilities.",
    "Question 15: I am more interested in the theoretical and abstract aspects of a situation.",
    "Question 16: I like to have a detailed plan before taking action.",
    "Question 17: We should make decisions based on logical reasoning and facts.",
    "Question 18: The best decision is the one that maximizes efficiency and productivity.",
    "Question 19: Objectivity should be the primary consideration when making decisions.",
    "Question 20: The most important factor in decision making is the analysis of available data.",
    "Question 21: I believe that we should prioritize our personal values in decision making.",
    "Question 22: I think we should consider how our actions will affect others before making a decision.",
    "Question 23: I feel that relationships and emotional connections should play a role in decision making.",
    "Question 24: I believe that intuition should guide our decisions in certain situations.",
    "Question 25: I like to have a clear plan and stick to it.",
    "Question 26: I enjoy having a structured and organized approach to tasks",
    "Question 27: I feel most comfortable when I know what to expect in a situation.",
    "Question 28: I believe in making decisions and sticking to them.",
    "Question 29: I prefer to stay flexible and adapt to changing situations.",
    "Question 30: I like to keep my options open and explore different possibilities.",
    "Question 31: I enjoy going with the flow and being spontaneous.",
    "Question 32: I think it's important to remain open to new ideas and possibilities."
]

# Initialize the user's answers list
user_answers = []

# Loop through the questions and ask for the user's answer
# Loop through the questions and ask for the user's answer
for question in questions:
    print(question)
    print("1. Strongly Disagree")
    print("2. Disagree")
    print("3. Neutral")
    print("4. Agree")
    print("5. Strongly Agree")
    user_answer = int(input("Your answer: "))
    user_answers.append(user_answer)
    print("\n")

print("Descriptive questions!\n")

# Define the questions and options
text_questions = [
    "Question 1: How do you prefer to spend your free time? ",
    "Question 2: How do you prefer to approach problem-solving? ",
    "Question 3: When making important decisions, how do you prioritize your values and beliefs? ",
    "Question 4: How do you typically approach your daily schedule and tasks? "
]
options = [
    ["I prefer to spend my free time alone, reading a book or watching a movie. I find it relaxing and rejuvenating to be in my own space and not have to socialize. I recharge my batteries by being alone.", "I prefer to spend my free time with others, going out with friends or trying new activities. I find it energizing and fun to be around people and have new experiences. I recharge my batteries by being with others."],
    ["I prefer to approach problem-solving by using my practical knowledge and experience. I like to focus on the concrete facts and details and come up with a solution that is based on tangible evidence. I find it more satisfying when I can find a solution that is tried and true.", "I prefer to approach problem-solving by using my imagination and creativity. I like to focus on the big picture and the possibilities, and come up with a solution that is unique and innovative. I find it more satisfying when I can find a solution that is novel and unexpected."],
    ["When making important decisions, I prioritize logic and objectivity. I try to consider the consequences of my actions objectively and make decisions based on what is fair and just. I find it more important to maintain a logical and consistent decision-making process, rather than basing my decisions on my emotions or personal values.", "When making important decisions, I prioritize my personal values and beliefs. I try to consider the impact of my actions on the people involved and make decisions based on what is in line with my heart. I find it more important to maintain a consistent set of values and beliefs, rather than basing my decisions on pure logic."],
    ["I prefer to have a structured and organized approach to my daily tasks. I like to have a plan for my day and follow it as closely as possible. I feel more comfortable when I have completed my tasks and my schedule is in order. I find that having a structured routine helps me to be more productive and achieve my goals.", "I prefer a flexible and spontaneous approach to my daily tasks. I like to leave room for surprises and unexpected opportunities. I feel more comfortable when I have the freedom to change my plans if something more interesting or exciting comes up. I find that having a flexible and spontaneous approach helps me to be more adaptable and go with the flow."]
]

# Initialize the user's answers list
text_answers = []

# Loop through the text_questions and ask for the user's answer
for i, question in enumerate(text_questions):
    print(question)
    print(f"Option 1: {options[i][0]}")
    print(f"Option 2: {options[i][1]}")
    text_answer = int(input("Your answer (1 or 2): "))
    text_answers.append(options[i][text_answer - 1])
    print("\n")

# Concatenate the selected options to form a single string
result = " ".join(text_answers)

# Initialize the TF-IDF Vectorizer with 32 features
tfidf = TfidfVectorizer(max_features=32)

# Fit and transform the textual data
result_tfidf = tfidf.fit_transform([result]).toarray()


# Append the textual questions answer to numerical questions answer 
user_answers.extend(result_tfidf.tolist())
print("user_answers: ",user_answers)


# extend the last list in the array to the existing list
user_answers.extend(user_answers.pop())

# print the updated user_answers array
print(user_answers)

# Convert the user's answers list to a numpy array
user_answers = np.array(user_answers).reshape(1, -1)

# Predict the user's personality type using the trained model
personality_type = model.predict(user_answers)

# Use the target_encoder to decode the encoded personality type
#personality_type = target_encoder.inverse_transform(encoded_personality_type)[0]

# Print the user's personality type
#print("Your personality type is:", personality_type)

Welcome to the personality quiz!

Question 1: I enjoy being in large groups of people.
1. Strongly Disagree
2. Disagree
3. Neutral
4. Agree
5. Strongly Agree


Question 2: I feel energized after interacting with many people.
1. Strongly Disagree
2. Disagree
3. Neutral
4. Agree
5. Strongly Agree


Question 3: I find it easy to approach and talk to strangers.
1. Strongly Disagree
2. Disagree
3. Neutral
4. Agree
5. Strongly Agree


Question 4: I like being the center of attention in social situations.
1. Strongly Disagree
2. Disagree
3. Neutral
4. Agree
5. Strongly Agree


Question 5: I prefer spending time alone over being with others.
1. Strongly Disagree
2. Disagree
3. Neutral
4. Agree
5. Strongly Agree


Question 6: I feel drained after spending a lot of time around people and need alone time to recharge.
1. Strongly Disagree
2. Disagree
3. Neutral
4. Agree
5. Strongly Agree


Question 7: I prefer to work independently and avoid group projects if possible.	
1. Strongly Disagree
2. Dis

In [7]:
# Print the user's personality type
print("Your personality type is:", personality_type)

# Convert the predicted class to an integer value
predicted_class = int(personality_type)

# Define a switch case function
def switch_case(x):
    switcher = {
        0: "ENFJ(The Teacher): Warm and empathetic, ENFJs are natural leaders who bring out the best in others. They are excellent communicators and know how to bring harmony to a group. ENFJs are driven by a sense of purpose and a desire to make a positive impact on the world.",
        1: "ENFP(The Champion): Enthusiastic and optimistic, ENFPs are full of ideas and always ready for an adventure. They are excellent at understanding and connecting with others, and enjoy bringing people together. ENFPs are highly creative and often excel at finding new and innovative solutions to problems.",
        2: "ENTJ(The Commander): Strategic and driven, ENTJs are natural leaders who excel at creating and executing plans. They are confident and assertive, and are not afraid to make tough decisions. ENTJs are highly ambitious and always looking for new challenges to conquer.",
        3: "ENTP(The Visionary): Quick-witted and always curious, ENTPs are natural problem-solvers who enjoy exploring new ideas and possibilities. They are enthusiastic and energetic, and are always looking for ways to shake things up. ENTPs are highly independent and value their freedom to think and act as they please.",
        4: "ESFJ(The Provider): Warm and supportive, ESFJs are natural caregivers who enjoy helping others. They are highly organized and reliable, and value traditions and stability. ESFJs are driven by a strong sense of responsibility and a desire to make a positive impact on those around them.",
        5: "ESFP(The Performer: Fun-loving and spontaneous, ESFPs are the life of the party and enjoy being in the spotlight. They are highly social and enjoy making connections with others. ESFPs live in the moment and value their freedom to enjoy life to the fullest.",
        6: "ESTJ(The Supervisor): Practical and down-to-earth, ESTJs are natural leaders who are highly organized and efficient. They value tradition and order, and are driven by a sense of duty and responsibility. ESTJs are highly assertive and confident, and are not afraid to take charge in a crisis.",
        7: "ESTP(The Dynamo): Bold and daring, ESTPs are natural risk-takers who enjoy living life on the edge. They are highly action-oriented and enjoy taking control of a situation. ESTPs value their independence and enjoy exploring new experiences and opportunities.",
        8: "INFJ(The Counselor): Intuitive and empathetic, INFJs are natural healers who enjoy helping others. They are highly imaginative and often have a strong sense of their life's purpose. INFJs are driven by a strong sense of morality and a desire to make a positive impact on the world.",
        9: "INFP(The Idealist): Idealistic and creative, INFPs are natural dreamers who enjoy exploring their own thoughts and feelings. They are highly empathetic and value deep connections with others. INFPs are driven by a strong sense of purpose and a desire to make a positive impact on the world.",
        10: "INTJ(The Architect): Strategic and visionary, INTJs are natural leaders who excel at developing and executing plans. They are highly analytical and enjoy finding new and innovative solutions to problems. INTJs are driven by a strong sense of purpose and a desire to make a positive impact on the world.",
        11: "INTP(The Architect): They are creative, innovative, and logical problem-solvers. They enjoy exploring new ideas and theories, and are not afraid to challenge traditional thinking.",
        12: "ISFJ(The Protector): They are responsible, reliable, and nurturing individuals. They have a strong sense of duty and put the needs of others before their own. They value tradition and stability.",
        13: "ISFP(The Composer): They are sensitive, imaginative, and spontaneous individuals. They are often artistic, and enjoy living in the moment. They value their independence and freedom.",
        14: "ISTJ(The Inspector): They are practical, dependable, and methodical individuals. They value order, stability, and structure, and are committed to their responsibilities. They are often the backbone of organizations.",
        15: "ISTP(The Craftsman): They are hands-on individuals who enjoy taking things apart and figuring out how they work. They are confident, independent, and resourceful, and value action and results over theoretical ideas."
    }
    return switcher.get(x, "Invalid class")

# Call the switch case function
result = switch_case(predicted_class)

# Print the result
print("Your personality type is", result)


Your personality type is: [[9]]
Your personality type is INFP(The Idealist): Idealistic and creative, INFPs are natural dreamers who enjoy exploring their own thoughts and feelings. They are highly empathetic and value deep connections with others. INFPs are driven by a strong sense of purpose and a desire to make a positive impact on the world.
